In [ ]:
# Install dependencies
%env TMPDIR=/home/newman/tmp
%pip install --cache-dir=/home/newman/tmp ultralytics

In [22]:
import logging
import requests
import zipfile
import shutil
import os

# Download dataset
logging.info('Downloading dataset...')
response = requests.get('https://www.kaggle.com/api/v1/datasets/download/stmlen/windturbine-damage-dataset-yolo-format')

# Save as .zip file
logging.info('Saving dataset...')
with open('./windturbine-damage-dataset-yolo-format.zip', 'wb') as f:
    f.write(response.content)

# Extract .zip
logging.info('Extracting dataset...')
with zipfile.ZipFile('./windturbine-damage-dataset-yolo-format.zip', 'r') as zip:
    zip.extractall('.')

# Clean unused files
os.remove('./windturbine-damage-dataset-yolo-format.zip')

In [21]:
import logging
import os
import random
import shutil

logging.info('Processing dataset...')

# Paths
source_images = 'data/NordTank1/images'
source_labels = 'data/NordTank1/labels'
dest_dir = 'dataset'

# Settings
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Create target folder structure
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(dest_dir, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dest_dir, split, 'labels'), exist_ok=True)

# Get all image filenames
image_files = [f for f in os.listdir(source_images) if f.endswith(('.jpg', '.png', '.jpeg'))]
random.shuffle(image_files)

# Split
total = len(image_files)
train_split = int(total * train_ratio)
val_split = int(total * (train_ratio + val_ratio))

splits = {
    'train': image_files[:train_split],
    'val': image_files[train_split:val_split],
    'test': image_files[val_split:]
}

for split, files in splits.items():
    for img_file in files:
        label_file = os.path.splitext(img_file)[0] + '.txt'

        shutil.copy(os.path.join(source_images, img_file), os.path.join(dest_dir, split, 'images', img_file))
        shutil.copy(os.path.join(source_labels, label_file), os.path.join(dest_dir, split, 'labels', label_file))

# Clean original dataset
shutil.rmtree('./data')

In [3]:
import ultralytics

ultralytics.checks()

Ultralytics 8.3.139 🚀 Python-3.10.15 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5747MiB)
Setup complete ✅ (12 CPUs, 31.3 GB RAM, 0.0/15.6 GB disk)


In [13]:
from ultralytics import YOLO

# Load a pretrained YOLO detection model
model = YOLO('yolo11n.pt')

In [14]:
# Train the model
model.train(data='./dataset/data.yaml', epochs=10)

Ultralytics 8.3.139 🚀 Python-3.10.15 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5747MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0,

train: Scanning /home/newman/Projects/wind-turbine-damage-recognition/dataset/train/labels.cache... 2096 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2096/2096 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1247.5±805.2 MB/s, size: 157.5 KB)


val: Scanning /home/newman/Projects/wind-turbine-damage-recognition/dataset/val/labels.cache... 599 images, 0 backgrounds, 0 corrupt: 100%|██████████| 599/599 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        599       1830      0.799      0.696      0.747      0.483



10 epochs completed in 0.113 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.10.15 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5747MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:03<00:00,  4.83it/s]


                   all        599       1830        0.8      0.693      0.747      0.484
                  dirt        119        129      0.909      0.845       0.88      0.646
                damage        498       1701      0.691      0.542      0.615      0.322
Speed: 0.4ms preprocess, 2.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ff336d7c610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [16]:
# Load the trained model
model = YOLO("./runs/detect/train/weights/best.pt")

# Predict on an image
model('./DJI_0008_04_04.png', save=True)


image 1/1 /home/newman/Projects/wind-turbine-damage-recognition/DJI_0008_04_04.png: 416x640 (no detections), 8.5ms
Speed: 1.5ms preprocess, 8.5ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'dirt', 1: 'damage'}
 obb: None
 orig_img: array([[[ 31,  68,  64],
         [ 33,  70,  66],
         [ 32,  73,  66],
         ...,
         [187, 182, 181],
         [188, 183, 182],
         [188, 183, 182]],
 
        [[ 34,  71,  67],
         [ 38,  75,  71],
         [ 33,  74,  67],
         ...,
         [186, 181, 180],
         [187, 182, 181],
         [188, 183, 182]],
 
        [[ 31,  68,  64],
         [ 37,  75,  69],
         [ 31,  72,  65],
         ...,
         [186, 181, 180],
         [187, 182, 181],
         [187, 182, 181]],
 
        ...,
 
        [[ 24,  49,  45],
         [ 27,  53,  47],
         [ 29,  55,  49],
         ...,
         [ 32,  69,  61],
         [ 36,  73,  65],
         [ 27,  64,  56]],
 
        [[ 24,  49,  45],
         [ 28,  54,  48],
         [ 27,  53,  47],
         ...,
       